# 🚀 ИСПРАВЛЕННЫЙ АВТОМАТИЧЕСКИЙ МУЛЬТИ-ТАЙМФРЕЙМ ОПТИМИЗАТОР

## 📋 **ИСПРАВЛЕННАЯ ВЕРСИЯ - РАБОТАЕТ С INDICATORENGINE**

Эта версия исправляет проблему с несуществующими классами индикаторов и правильно работает с `IndicatorEngine`.


In [ ]:
# 📦 ИМПОРТЫ И ЗАГРУЗКА ЗАВИСИМОСТЕЙ

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import os
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Загружаем конфигурацию и новый движок индикаторов
%run 01_config.ipynb
%run 08_indicator_engine_clean.ipynb

print("✅ Зависимости загружены!")
print("📊 Готов к автоматической оптимизации всех временных рамок")
print("🏗️ Используем исправленный модульный движок индикаторов")


In [ ]:
# 🏗️ ИСПРАВЛЕННЫЙ КЛАСС АВТОМАТИЧЕСКОГО ОПТИМИЗАТОРА

class FixedAutoMultiTimeframeOptimizer:
    """Исправленный автоматический оптимизатор - работает с IndicatorEngine"""
    
    def __init__(self):
        self.results = {}
        self.summary = {}
        
        # Создаем папку для результатов
        self.results_folder = "optimization_results"
        os.makedirs(self.results_folder, exist_ok=True)
        
        # ИСПРАВЛЕНИЕ: Используем функции индикаторов вместо классов
        self.available_indicators = {
            # Трендовые индикаторы
            'supertrend': {
                'name': 'SuperTrend',
                'function': 'calculate_supertrend',
                'params': ['atr_period', 'atr_multiplier'],
                'param_ranges': {
                    'atr_period': [5, 7, 10, 12, 15, 18, 20, 25, 30],
                    'atr_multiplier': [1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0]
                }
            },
            'rsi': {
                'name': 'RSI',
                'function': 'calculate_rsi',
                'params': ['length'],  # Убираем source, он передается отдельно
                'param_ranges': {
                    'length': [8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
                }
            },
            'macd': {
                'name': 'MACD',
                'function': 'calculate_macd',
                'params': ['fast_period', 'slow_period', 'signal_period'],
                'param_ranges': {
                    'fast_period': [5, 8, 10, 12, 15, 18, 20, 24, 26, 30],
                    'slow_period': [15, 20, 25, 30, 35, 40, 45, 50, 60, 70],
                    'signal_period': [5, 7, 9, 12, 15, 18, 21, 24, 27, 30]
                }
            },
            'bollinger_bands': {
                'name': 'Bollinger Bands',
                'function': 'calculate_bollinger_bands',
                'params': ['period', 'std_dev'],
                'param_ranges': {
                    'period': [15, 20, 25, 30, 35, 40],
                    'std_dev': [1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
                }
            }
        }
        
        # Файлы данных для разных временных рамок
        self.data_files = {
            '15m': 'df_btc_15m.csv',
            '30m': 'df_btc_30m.csv', 
            '1h': 'df_btc_1h.csv',
            '4h': 'df_btc_4h.csv',
            '1d': 'df_btc_1d.csv'
        }
        
        print("✅ FixedAutoMultiTimeframeOptimizer инициализирован!")
        print(f"📁 Папка результатов: {self.results_folder}")
        print(f"🎯 Доступные индикаторы: {len(self.available_indicators)}")
        print(f"📊 Доступные временные рамки: {list(self.data_files.keys())}")

print("✅ Исправленный класс AutoMultiTimeframeOptimizer создан!")


In [ ]:
# 🔧 МЕТОДЫ ИСПРАВЛЕННОГО ОПТИМИЗАТОРА

def load_data(self, timeframe):
    """Загружает данные для указанной временной рамки"""
    filename = self.data_files.get(timeframe)
    if not filename:
        print(f"❌ Неизвестная временная рамка: {timeframe}")
        return None
    
    try:
        data = pd.read_csv(filename, index_col=0, parse_dates=True)
        print(f"✅ Загружены данные для {timeframe}: {len(data)} записей")
        return data
    except Exception as e:
        print(f"❌ Ошибка загрузки {filename}: {e}")
        return None

def optimize_indicator_for_timeframe(self, indicator_key, timeframe, max_iterations=20):
    """Оптимизирует конкретный индикатор для временной рамки"""
    print(f"\n🔍 ОПТИМИЗАЦИЯ {indicator_key.upper()} для {timeframe.upper()}")
    print("-" * 60)
    
    # Загружаем данные
    data = self.load_data(timeframe)
    if data is None:
        return None
    
    # Создаем движок индикаторов
    engine = IndicatorEngine(data)
    
    # Получаем конфигурацию индикатора
    indicator_config = self.available_indicators.get(indicator_key)
    if not indicator_config:
        print(f"❌ Неизвестный индикатор: {indicator_key}")
        return None
    
    function_name = indicator_config['function']
    param_ranges = indicator_config['param_ranges']
    
    # Проверяем, что функция существует в движке
    if not hasattr(engine, function_name):
        print(f"❌ Функция {function_name} не найдена в IndicatorEngine")
        return None
    
    print(f"📊 Функция: {function_name}")
    print(f"📋 Параметры: {list(param_ranges.keys())}")
    
    best_result = None
    best_score = -1
    iteration = 0
    
    # Простая оптимизация - перебираем комбинации параметров
    import itertools
    
    # Создаем сетку параметров
    param_names = list(param_ranges.keys())
    param_values = list(param_ranges.values())
    
    # Ограничиваем количество комбинаций
    max_combinations = max_iterations
    combinations = list(itertools.product(*param_values))[:max_combinations]
    
    print(f"🔄 Тестируем {len(combinations)} комбинаций параметров...")
    
    for param_combo in combinations:
        iteration += 1
        params = dict(zip(param_names, param_combo))
        
        try:
            # Вызываем функцию индикатора
            if function_name == 'calculate_rsi':
                # Для RSI нужен source как серия данных, но не передаем его в params
                rsi_params = params.copy()
                if 'source' in rsi_params:
                    del rsi_params['source']  # Удаляем source из params
                result = getattr(engine, function_name)(source=data['close'], **rsi_params)
            else:
                result = getattr(engine, function_name)(**params)
            
            # Оценка качества результата
            if function_name == 'calculate_rsi':
                # Для RSI - баланс сигналов
                buy_signals = ((result < 30) & (result.shift(1) >= 30)).sum()
                sell_signals = ((result > 70) & (result.shift(1) <= 70)).sum()
                total_signals = buy_signals + sell_signals
                
                if total_signals > 0:
                    balance = abs(buy_signals - sell_signals) / total_signals
                    quality_score = 1 - balance
                else:
                    quality_score = 0
                    
            elif function_name == 'calculate_supertrend':
                # Для SuperTrend - количество изменений тренда
                if isinstance(result, tuple):
                    st = result[0]
                else:
                    st = result
                    
                trend_changes = ((st != st.shift(1)) & (st.notna())).sum()
                quality_score = min(trend_changes / 100, 1.0)
                
            else:
                # Для других индикаторов - простая оценка
                if isinstance(result, (pd.Series, pd.DataFrame)):
                    quality_score = 0.5  # Базовая оценка
                else:
                    quality_score = 0.3
            
            # Проверяем, является ли это лучшим результатом
            if quality_score > best_score:
                best_score = quality_score
                best_result = {
                    'indicator': indicator_key,
                    'timeframe': timeframe,
                    'parameters': params,
                    'score': quality_score,
                    'function': function_name,
                    'iteration': iteration
                }
            
            if iteration % 5 == 0:
                print(f"   Итерация {iteration:3d}: {params} -> {quality_score:.3f}")
                
        except Exception as e:
            print(f"   Итерация {iteration:3d}: ошибка - {e}")
            continue
    
    if best_result:
        print(f"\n🏆 ЛУЧШИЙ РЕЗУЛЬТАТ:")
        print(f"   Параметры: {best_result['parameters']}")
        print(f"   Оценка: {best_result['score']:.4f}")
        print(f"   Итерация: {best_result['iteration']}")
    
    return best_result

# Добавляем методы к классу
FixedAutoMultiTimeframeOptimizer.load_data = load_data
FixedAutoMultiTimeframeOptimizer.optimize_indicator_for_timeframe = optimize_indicator_for_timeframe

print("✅ Методы исправленного оптимизатора добавлены!")


In [1]:
# 🚀 ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО ОПТИМИЗАТОРА

print("🎯 ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО ОПТИМИЗАТОРА")
print("=" * 60)

# Создаем исправленный оптимизатор
fixed_optimizer = FixedAutoMultiTimeframeOptimizer()

print("\n🔍 ТЕСТИРУЕМ ОПТИМИЗАЦИЮ RSI для 1H:")
print("-" * 50)

# Тестируем оптимизацию RSI для 1H
rsi_result = fixed_optimizer.optimize_indicator_for_timeframe('rsi', '1h', max_iterations=10)

if rsi_result:
    print(f"\n✅ ОПТИМИЗАЦИЯ RSI ЗАВЕРШЕНА!")
    print(f"🏆 Лучший результат:")
    print(f"   Временная рамка: {rsi_result['timeframe']}")
    print(f"   Параметры: {rsi_result['parameters']}")
    print(f"   Оценка: {rsi_result['score']:.4f}")
else:
    print("❌ Ошибка оптимизации RSI")

print(f"\n🎉 ИСПРАВЛЕННЫЙ ОПТИМИЗАТОР РАБОТАЕТ!")
print(f"💡 Теперь нет ошибок с несуществующими классами!")


🎯 ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО ОПТИМИЗАТОРА


NameError: name 'FixedAutoMultiTimeframeOptimizer' is not defined